<a href="https://colab.research.google.com/github/haleemaraza/agent_sdk_context/blob/main/context.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [13]:
!pip install -Uq openai-agents

In [14]:
import nest_asyncio
nest_asyncio.apply()

In [15]:
from pydantic import BaseModel
from agents import AsyncOpenAI, OpenAIChatCompletionsModel, RunConfig
from google.colab import userdata

In [20]:
google_api_key= userdata.get("GOOGLE_API_KEY")

if not google_api_key:
  raise ValueError("GOOGLE_API_KEY please ensure itis defined in your .env file.")

external_client=AsyncOpenAI(
    api_key=google_api_key,
    base_url="https://generativelanguage.googleapis.com/v1beta/openai",
)

model= OpenAIChatCompletionsModel(
    model="gemini-2.0-flash",
    openai_client=external_client
)


In [21]:
from agents import set_tracing_disabled
set_tracing_disabled(True)

In [23]:
import asyncio
from dataclasses import dataclass
from agents import Agent, RunContextWrapper,Runner, function_tool

@dataclass
class UserInfo:
  name:str
  uid:int
  age:int
  location:str="Pakistan"

@function_tool
async def fetch_user_age(wrapper:RunContextWrapper[UserInfo]) -> str:
    '''Returns the age of the user'''
    print("[->Tool]",wrapper,"\n\n")
    return f"User {wrapper.context.name} is {wrapper.context.age} years old"

@function_tool
async def fetch_user_location(wrapper:RunContextWrapper[UserInfo]) -> str:
    '''Returns the current location of the user'''
    print("[->Tool Current location]",wrapper,"\n\n")
    return f"User {wrapper.context.name} is the current at {wrapper.context.location}"

async def main():
  user_info=UserInfo(name="Arfa", uid=28, age=18)

  agent =Agent[UserInfo](
      name="Assistant",
      tools=[fetch_user_age,fetch_user_location],
      model=model
  )

  result= await Runner.run(
      starting_agent=agent,
      input="what is the age and location of the user?",
      context=user_info,
  )
  print(result.final_output)

if __name__ == "__main__":
    asyncio.run(main())

[->Tool] ToolContext(context=UserInfo(name='Arfa', uid=28, age=18, location='Pakistan'), usage=Usage(requests=1, input_tokens=33, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=10, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=43), tool_name='fetch_user_age', tool_call_id='') 


[->Tool Current location] ToolContext(context=UserInfo(name='Arfa', uid=28, age=18, location='Pakistan'), usage=Usage(requests=1, input_tokens=33, input_tokens_details=InputTokensDetails(cached_tokens=0), output_tokens=10, output_tokens_details=OutputTokensDetails(reasoning_tokens=0), total_tokens=43), tool_name='fetch_user_location', tool_call_id='') 


The user is 18 years old and is currently located in Pakistan.

